In [3]:
import numpy as np
import pandas as pd
import os
import math as m
import matplotlib.pyplot as plt
import time

## Import Data Functions

In [4]:
df = pd.read_csv(r"..\data\dataset_new_score_v3.csv").drop(columns="Unnamed: 0")
df.columns

Index(['titre', 'budget', 'recette', 'duree', 'realisateur', 'casting',
       'compagnies_production', 'suite', 'Comedy', 'Drama', 'Horror',
       'Thriller', 'Action', 'Adventure', 'Science Fiction', 'Fantasy',
       'Family', 'Crime', 'Romance', 'Animation', 'Mystery', 'War', 'Music',
       'Western', 'History', 'Documentary', 'TV Movie', 'benefice global',
       'saison', 'casting_score_sum', 'casting_score_mean',
       'casting_score_med', 'casting_score_min', 'casting_score_max',
       'director_score_sum', 'director_score_mean', 'director_score_med',
       'director_score_min', 'director_score_max', 'compagnies_score_sum',
       'compagnies_score_mean', 'compagnies_score_med', 'compagnies_score_min',
       'compagnies_score_max'],
      dtype='object')

## Datasets and Scaling

In [5]:
from sklearn.model_selection import train_test_split

TEST_RATIO = 0.9
 
X_train, X_test, Y_train, Y_test = train_test_split(df.drop(columns=["titre", "recette", "realisateur", "casting", "compagnies_production", "benefice global"]), df["benefice global"], test_size = TEST_RATIO, shuffle=True, random_state = 1)

In [6]:
from sklearn.preprocessing import StandardScaler

X_train_f = StandardScaler().fit(np.array(X_train)).transform(np.array(X_train))


X_test_f = StandardScaler().fit(np.array(X_test)).transform(np.array(X_test))

print("Done")

Done


## Model

In [7]:
from sklearn import svm

start = time.time()
regr = svm.SVR(gamma=0.01, C=10)
regr.fit(X_train_f, Y_train)
end = time.time()
elapsed1 = end - start

print("Temps exécution : ", elapsed1)

Temps exécution :  0.026971817016601562


In [8]:
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import mean_absolute_error

#Get the time of execution
start = time.time()


score_mape = mean_absolute_percentage_error(Y_test, regr.predict(X_test_f))
score_mae = mean_absolute_error(Y_test, regr.predict(X_test_f))


end = time.time()
elapsed2 = elapsed1 + end - start

print("Temps exécution : ", elapsed2)
print("MAPE : ", score_mape)
print("MAE : ", score_mae)

Temps exécution :  0.9149565696716309
MAPE :  2429046475978963.5
MAE :  230.42393792958546


In [9]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

#Get the time of execution

scorer = make_scorer(mean_absolute_error)
score = cross_val_score(regr, X_train_f, Y_train, cv=10, scoring=scorer)


print("%0.2f MAE with a standard deviation of %0.2f" % (score.mean(), score.std()))




scorer = make_scorer(mean_absolute_percentage_error)
score = cross_val_score(regr, X_train_f, Y_train, cv=10, scoring=scorer)



print("%0.2f MAPE with a standard deviation of %0.2f" % (score.mean(), score.std()))


246.02 MAE with a standard deviation of 86.71
771167787537418.00 MAPE with a standard deviation of 2313503362612242.50


### GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

C = [0.1,0.5,1,2,3]
gamma = [0.1, 0.2,0.3]

params_grid = {'C': C, 'gamma': gamma}

rf_clf = svm.SVR()

rf_cv = GridSearchCV(rf_clf, params_grid, scoring="neg_mean_absolute_error", cv=10)
rf_cv.fit(X_train, Y_train)


print('GridSearch CV best score : {:.4f}\n\n'.format(rf_cv.best_score_))

# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (rf_cv.best_params_))

# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (rf_cv.best_estimator_))